In [18]:
from gurobipy import GRB, Model, quicksum
import gurobipy as gb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [2]:
heat = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/heat.csv')
lubricant_materials = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_materials.csv')
lubricant_products = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_products.csv')
sustainability = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sustainability.csv')
welders = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/welders_data_makeup.csv')

# Question 1

In [24]:
sustainability.columns

Index(['Unnamed: 0', 'Material_1', 'Material_2', 'Material_3', 'Material_4',
       'Material_5', 'Material_6', 'Material_7', 'Material_8', 'Material_9',
       ...
       'Material_111', 'Material_112', 'Material_113', 'Material_114',
       'Material_115', 'Material_116', 'Material_117', 'Material_118',
       'Material_119', 'Material_120'],
      dtype='object', length=121)

In [25]:
heat.columns

Index(['Unnamed: 0', 'Material_1', 'Material_2', 'Material_3', 'Material_4',
       'Material_5', 'Material_6', 'Material_7', 'Material_8', 'Material_9',
       ...
       'Material_111', 'Material_112', 'Material_113', 'Material_114',
       'Material_115', 'Material_116', 'Material_117', 'Material_118',
       'Material_119', 'Material_120'],
      dtype='object', length=121)

In [26]:
lubricant_materials.columns

Index(['max_availability', 'p_max'], dtype='object')

In [28]:
lubricant_products.columns

Index(['revenue_per_unit', 'total_demand', 'max_heat', 'min_sustainability'], dtype='object')

In [3]:
sustainability.head()

,Unnamed: 0,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Material_7,Material_8,Material_9,...,Material_111,Material_112,Material_113,Material_114,Material_115,Material_116,Material_117,Material_118,Material_119,Material_120
0,Lubricant_1,0.147829,0.039705,0.069833,0.039332,0.067766,0.190804,0.199973,0.095008,0.054476,...,0.131623,0.033194,0.093441,0.087834,0.094649,0.165678,0.030530,0.190125,0.066358,0.051525
1,Lubricant_2,0.182291,0.083269,0.098084,0.164984,0.040019,0.133258,0.077280,0.165286,0.024031,...,0.036514,0.084092,0.147768,0.053151,0.071207,0.142609,0.181199,0.101304,0.067911,0.144488
2,Lubricant_3,0.083250,0.113593,0.175466,0.138051,0.111207,0.051333,0.081790,0.143911,0.186332,...,0.179769,0.085461,0.159866,0.139562,0.113184,0.196185,0.131290,0.141619,0.157380,0.104129
3,Lubricant_4,0.141885,0.155877,0.102057,0.064875,0.156477,0.096769,0.172445,0.089421,0.133271,...,0.163672,0.151913,0.082432,0.178005,0.033005,0.157605,0.124326,0.194770,0.142742,0.162807
4,Lubricant_5,0.159210,0.180234,0.190785,0.164466,0.056306,0.167539,0.147017,0.158347,0.180348,...,0.087475,0.055186,0.035480,0.057400,0.080102,0.107849,0.073504,0.129936,0.045869,0.150771


In [4]:
heat.head()

,Unnamed: 0,Material_1,Material_2,Material_3,Material_4,Material_5,Material_6,Material_7,Material_8,Material_9,...,Material_111,Material_112,Material_113,Material_114,Material_115,Material_116,Material_117,Material_118,Material_119,Material_120
0,Lubricant_1,0.088917,0.059716,0.036411,0.011631,0.060158,0.029712,0.056947,0.068439,0.028212,...,0.093050,0.046852,0.060321,0.016563,0.071261,0.025915,0.062219,0.062740,0.057524,0.097754
1,Lubricant_2,0.019742,0.013647,0.020847,0.067725,0.088228,0.090721,0.099536,0.041835,0.014235,...,0.097604,0.039395,0.068045,0.011744,0.037225,0.046810,0.052762,0.092303,0.072814,0.041260
2,Lubricant_3,0.070982,0.053853,0.018249,0.011367,0.029872,0.038974,0.025282,0.020939,0.079439,...,0.070471,0.061922,0.072262,0.081593,0.074708,0.044338,0.057732,0.054325,0.031015,0.054989
3,Lubricant_4,0.080620,0.079541,0.065975,0.047528,0.023671,0.044788,0.065799,0.024638,0.017213,...,0.096655,0.030953,0.030881,0.019204,0.088607,0.091777,0.068694,0.064566,0.035833,0.058934
4,Lubricant_5,0.065586,0.035383,0.071708,0.026711,0.067208,0.094627,0.038795,0.022624,0.031238,...,0.023396,0.062185,0.061785,0.075099,0.049962,0.095335,0.032829,0.054314,0.012586,0.059446


In [5]:
lubricant_materials.head()

,max_availability,p_max
0,3943.686556,0.367249
1,6000.990285,0.389034
2,6186.812534,0.337231
3,7075.045587,0.321033
4,1077.332197,0.381319


In [6]:
lubricant_products.head()

,revenue_per_unit,total_demand,max_heat,min_sustainability
0,41.416945,31442.462706,12.622450,55.821198
1,64.418807,41239.237566,11.845832,67.120316
2,72.961810,39274.924679,11.594659,68.968516
3,79.619072,31917.265523,10.611528,63.997325
4,73.372665,34287.165364,13.671678,68.812814


In [35]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
try:
    sustainability = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sustainability.csv')
    heat = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/heat.csv')
    lubricant_materials = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_materials.csv')
    lubricant_products = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_products.csv')
except Exception as e:
    print(f"Error loading data: {e}")
    exit()

# Model dimensions
num_lubricants = len(lubricant_products)
num_materials = len(lubricant_materials)

def create_linear_model(sustainability, heat, lubricant_materials, lubricant_products, with_regulatory_constraints=True):
    """
    Create the linear programming model for KryptoFlow
    
    Parameters:
    sustainability, heat, lubricant_materials, lubricant_products: DataFrames containing problem data
    with_regulatory_constraints (bool): Whether to include regulatory constraints
    
    Returns:
    model: Gurobi model object
    x: Dictionary of decision variables
    production: Dictionary of production quantity variables
    """
    # Create a new model
    model = gp.Model("KryptoFlow_Production")

    # Decision Variables: x[i,j] = amount of material j used in lubricant i
    x = model.addVars(num_lubricants, num_materials, lb=0, vtype=GRB.CONTINUOUS, name="x")
    production = model.addVars(num_lubricants, lb=0, vtype=GRB.CONTINUOUS, name="production")

    # Objective: Maximize total revenue
    model.setObjective(
        gp.quicksum(lubricant_products.loc[i, 'revenue_per_unit'] * production[i] for i in range(num_lubricants)),
        GRB.MAXIMIZE
    )

    # Constraint 1: Demand constraints
    for i in range(num_lubricants):
        model.addConstr(
            production[i] <= lubricant_products.loc[i, 'total_demand'],
            name=f"demand_{i}"
        )

    # Constraint 2: Material availability
    for j in range(num_materials):
        model.addConstr(
            gp.quicksum(x[i, j] for i in range(num_lubricants)) <= lubricant_materials.loc[j, 'max_availability'],
            name=f"material_avail_{j}"
        )

    # Constraint 3: Maximum proportion of each material in each lubricant
    for i in range(num_lubricants):
        for j in range(num_materials):
            model.addConstr(
                x[i, j] <= lubricant_materials.loc[j, 'p_max'] * production[i],
                name=f"max_prop_{i}_{j}"
            )

    if with_regulatory_constraints:
        # Constraint 4: Sustainability requirements for each lubricant
        for i in range(num_lubricants):
            model.addConstr(
                gp.quicksum(sustainability.loc[i, f'Material_{j+1}'] * x[i, j] for j in range(num_materials)) >= 
                lubricant_products.loc[i, 'min_sustainability'] * production[i],
                name=f"min_sustainability_{i}"
            )

        # Constraint 5: Heat limitations for each lubricant
        for i in range(num_lubricants):
            model.addConstr(
                gp.quicksum(heat.loc[i, f'Material_{j+1}'] * x[i, j] for j in range(num_materials)) <= 
                lubricant_products.loc[i, 'max_heat'] * production[i],
                name=f"max_heat_{i}"
            )

        # Constraint 6: Total sustainability across all products
        model.addConstr(
            gp.quicksum(sustainability.loc[i, f'Material_{j+1}'] * x[i, j] for i in range(num_lubricants) for j in range(num_materials)) >= 
            2 * gp.quicksum(lubricant_products.loc[i, 'min_sustainability'] * production[i] for i in range(num_lubricants)),
            name="total_sustainability"
        )

    return model, x, production

def solve_kryptoflow_problem():
    """Solve the KryptoFlow optimization problem and answer all exam questions"""
    
    # (a) Decision Variables
    print("\n(a) Decision Variables:")
    print("The decision variables x[i,j] represent the amount of material j used in lubricant i.")
    print(f"Total number of decision variables: {num_lubricants} lubricants × {num_materials} materials = {num_lubricants * num_materials}")
    
    # Build and solve the linear model
    linear_model, x_linear, production_linear = create_linear_model(
        sustainability, heat, lubricant_materials, lubricant_products
    )
    linear_model.optimize()
    
    # (b) Optimal Revenue
    print("\n(b) Optimal Revenue:")
    if linear_model.status == GRB.OPTIMAL:
        print(f"The optimal revenue is ${linear_model.objVal:.2f}")
    else:
        print("Linear model could not be solved to optimality.")

    # (c) Are there any lubricant products that are not produced?
    print("\n(c) Production Analysis:")
    not_produced = []
    
    for i in range(num_lubricants):
        if production_linear[i].X < 1e-5:
            not_produced.append(i+1)  # 1-indexed for reporting
    
    if not_produced:
        print(f"{len(not_produced)} lubricants are not produced: {not_produced}")
    else:
        print("All lubricant products are produced.")

    # (f) Solve model without regulatory constraints
    print("\n(f) Model without Regulatory Constraints:")
    unreg_model, _, _ = create_linear_model(sustainability, heat, lubricant_materials, lubricant_products, with_regulatory_constraints=False)
    unreg_model.optimize()
    
    if linear_model.status == GRB.OPTIMAL and unreg_model.status == GRB.OPTIMAL:
        diff = unreg_model.objVal - linear_model.objVal
        
        print(f"Optimal revenue without regulatory constraints: ${unreg_model.objVal:.2f}")
        
        if linear_model.objVal != 0:  # Prevent division by zero
            percentage_change = (diff / linear_model.objVal) * 100
            print(f"Difference from regulated model: ${diff:.2f} (+{percentage_change:.2f}%)")
        else:
            print("Difference from regulated model: ${diff:.2f} (Linear model objective value is 0, percentage change not computed)")
    else:
        print("One or both models could not be solved to optimality.")

if __name__ == "__main__":
    solve_kryptoflow_problem()



(a) Decision Variables:
The decision variables x[i,j] represent the amount of material j used in lubricant i.
Total number of decision variables: 60 lubricants × 120 materials = 7200
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7501 rows, 7260 columns and 43440 nonzeros
Model fingerprint: 0x9b3842d9
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [4e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 4e+04]
Presolve removed 60 rows and 0 columns
Presolve time: 0.01s
Presolved: 7441 rows, 7260 columns, 43380 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 60
 AA' NZ     : 5.088e+04
 Factor NZ  : 8.865e+04 (roughly 7 MB of memory)
 Factor Ops : 9.395e+06 (less than 1 se

In [49]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

def main():
    # -------------------------------------------------------------------
    # 1. Load Data
    # -------------------------------------------------------------------
    # These CSVs must match the structure you shared:
    #   - sustainability: rows = Lubricant_i, columns = Material_j
    #   - heat: same shape as sustainability
    #   - lubricant_materials: rows = Material_j, columns = [max_availability, p_max]
    #   - lubricant_products: rows = Lubricant_i, columns = [revenue_per_unit, total_demand, max_heat, min_sustainability]
    # Each has an "Unnamed: 0" column with either "Lubricant_i" or "Material_j" labels.
    sustainability = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sustainability.csv')
    heat = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/heat.csv')
    lubricant_materials = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_materials.csv')
    lubricant_products = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_products.csv')
    
    # Set the index to the "Unnamed: 0" column for easier lookups
    sustainability.set_index('Unnamed: 0', inplace=True)
    heat.set_index('Unnamed: 0', inplace=True)
    lubricant_materials.set_index('Unnamed: 0', inplace=True)
    lubricant_products.set_index('Unnamed: 0', inplace=True)

    # Build lists of lubricants and materials from the DataFrame indexes
    lubricants = list(lubricant_products.index)   # e.g. "Lubricant_1" ... "Lubricant_60"
    materials = list(lubricant_materials.index)   # e.g. "Material_1"  ... "Material_120"
    
    # -------------------------------------------------------------------
    # 2. Create Gurobi Model
    # -------------------------------------------------------------------
    m = Model("KryptoFlow_Production")
    # Uncomment to silence solver output: m.setParam('OutputFlag', 0)
    
    # -------------------------------------------------------------------
    # 3. Decision Variables
    # -------------------------------------------------------------------
    # production[i] = total production of lubricant i
    production = {}
    # x[i,j] = amount of material j used in lubricant i
    x = {}
    
    for i in lubricants:
        # Demand limit for each lubricant
        demand_i = lubricant_products.loc[i, "total_demand"]
        production[i] = m.addVar(lb=0, ub=demand_i, name=f"prod_{i}")
        
        for j in materials:
            x[i, j] = m.addVar(lb=0, name=f"x_{i}_{j}")
    
    m.update()
    
    # -------------------------------------------------------------------
    # 4. Constraints
    # -------------------------------------------------------------------
    
    # (a) Production = sum of materials used
    for i in lubricants:
        m.addConstr(
            production[i] == quicksum(x[i, j] for j in materials),
            name=f"link_{i}"
        )
    
    # (b) Material availability: sum of x[i,j] over i <= max_availability_j
    for j in materials:
        max_avail_j = lubricant_materials.loc[j, "max_availability"]
        m.addConstr(
            quicksum(x[i, j] for i in lubricants) <= max_avail_j,
            name=f"avail_{j}"
        )
    
    # (c) Proportion constraint: x[i,j] <= p_max_j * production[i]
    for j in materials:
        pmax_j = lubricant_materials.loc[j, "p_max"]
        for i in lubricants:
            m.addConstr(
                x[i, j] <= pmax_j * production[i],
                name=f"pmax_{i}_{j}"
            )
    
    # (d) Heat constraint: sum_j(heat[i,j] * x[i,j]) <= max_heat[i] * production[i]
    for i in lubricants:
        max_heat_i = lubricant_products.loc[i, "max_heat"]
        m.addConstr(
            quicksum(heat.loc[i, j] * x[i, j] for j in materials) <= max_heat_i * production[i],
            name=f"heat_{i}"
        )
    
    # (e) Sustainability constraint: sum_j(sustainability[i,j] * x[i,j]) >= min_sustainability[i] * production[i]
    for i in lubricants:
        min_sust_i = lubricant_products.loc[i, "min_sustainability"]
        m.addConstr(
            quicksum(sustainability.loc[i, j] * x[i, j] for j in materials) >= min_sust_i * production[i],
            name=f"sustain_{i}"
        )
    
    # -------------------------------------------------------------------
    # 5. Objective: Maximize total revenue
    # -------------------------------------------------------------------
    m.setObjective(
        quicksum(lubricant_products.loc[i, "revenue_per_unit"] * production[i] for i in lubricants),
        GRB.MAXIMIZE
    )
    
    # -------------------------------------------------------------------
    # 6. Solve
    # -------------------------------------------------------------------
    m.optimize()
    
    # -------------------------------------------------------------------
    # 7. Report Results
    # -------------------------------------------------------------------
    if m.status == GRB.OPTIMAL:
        print("\n=== Optimization Results ===")
        print(f"Optimal Total Revenue: ${m.objVal:,.2f}\n")
        
        # Collect production levels
        prod_solution = {i: production[i].X for i in lubricants}
        
        # Print a few lines for each lubricant
        for i in lubricants:
            print(f"{i} -> Production: {prod_solution[i]:.2f} / Demand: {lubricant_products.loc[i, 'total_demand']:.2f}")
        
        # Identify lubricants not produced (near zero production)
        not_produced = [i for i in lubricants if prod_solution[i] < 1e-6]
        if not_produced:
            print(f"\nLubricants not produced (production < 1e-6): {not_produced}")
        else:
            print("\nAll lubricants have positive production.")
        
        # Example: Shadow price for Material_27
        # If it exists, we can query the dual value of the "avail_Material_27" constraint
        mat_27_constr = m.getConstrByName("avail_Material_27")
        if mat_27_constr is not None:
            shadow_price_27 = mat_27_constr.Pi
            print(f"\nShadow price for Material_27: ${shadow_price_27:.4f} per unit")
            print(f"KryptoFlow would pay up to ${shadow_price_27 * 5:.4f} for 5 extra units of Material_27.\n")
        else:
            print("\nConstraint for Material_27 not found (check your naming).")
    else:
        print(f"Model did not solve to optimality. Status code: {m.status}")

if __name__ == "__main__":
    main()


KeyError: "None of ['Unnamed: 0'] are in the columns"

In [50]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# -------------------------------------------------------------------------
# 1. LOAD THE DATA
# -------------------------------------------------------------------------
heat = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/heat.csv')
sustainability = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/sustainability.csv')
lubricant_materials = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_materials.csv')
lubricant_products = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/lubricant_products.csv')

# The 'heat' and 'sustainability' data frames each have 60 rows (for 60 lubricants)
# and 121 columns of material data (plus one 'Unnamed: 0' column labeling rows).
# The 'lubricant_materials' data frame has 121 rows (one per material)
# with columns: ['Unnamed: 0','max_availability','p_max'].
# The 'lubricant_products' data frame has 60 rows (one per lubricant)
# with columns: ['Unnamed: 0','revenue_per_unit','total_demand','max_heat','min_sustainability'].

# -------------------------------------------------------------------------
# 2. REFORMAT FOR EASY LOOKUPS
# -------------------------------------------------------------------------

# Lubricant list (e.g., 'Lubricant_1', 'Lubricant_2', ...)
lubricants = list(lubricant_products['Unnamed: 0'])
# Material list (e.g., 'Material_1', 'Material_2', ...)
materials = list(lubricant_materials['Unnamed: 0'])

# Create dictionaries for quick parameter lookup by lubricant or material
revenue      = dict(zip(lubricant_products['Unnamed: 0'], lubricant_products['revenue_per_unit']))
demand       = dict(zip(lubricant_products['Unnamed: 0'], lubricant_products['total_demand']))
max_heat     = dict(zip(lubricant_products['Unnamed: 0'], lubricant_products['max_heat']))
min_sustain  = dict(zip(lubricant_products['Unnamed: 0'], lubricant_products['min_sustainability']))

max_avail    = dict(zip(lubricant_materials['Unnamed: 0'], lubricant_materials['max_availability']))
p_max        = dict(zip(lubricant_materials['Unnamed: 0'], lubricant_materials['p_max']))

# For heat and sustainability, convert them to dictionaries keyed by (lubricant, material).
# The CSV has each row = lubricant, each column = material_1, material_2, ...
# We'll skip column 0 ('Unnamed: 0') and row 0 if needed.

heat_dict = {}
sus_dict  = {}

# Ensure row labels in 'heat' and 'sustainability' match your 'lubricants' list
# Typically, row i in these data frames corresponds to lubricants[i].
# We assume the index in the data frames is aligned with the order in 'lubricants'.

heat.set_index('Unnamed: 0', inplace=True)         # row labels = 'Lubricant_1', etc.
sustainability.set_index('Unnamed: 0', inplace=True)

for lub in lubricants:
    for mat in materials:
        heat_dict[(lub, mat)] = heat.loc[lub, mat]
        sus_dict[(lub, mat)]  = sustainability.loc[lub, mat]

# -------------------------------------------------------------------------
# 3. BUILD THE MODEL
# -------------------------------------------------------------------------
model = gp.Model("KryptoFlow_Production")

# Decision variables: x_{i,j} = amount of material j used in lubricant i
x = model.addVars(lubricants, materials, lb=0, vtype=GRB.CONTINUOUS, name="x")

# -------------------------------------------------------------------------
# 4. OBJECTIVE: Maximize total revenue
# -------------------------------------------------------------------------
# If each "unit" of lubricant i is sum_j x_{i,j}, then total units of i is sum_j x_{i,j}.
# Revenue = sum_i (revenue_i * total_units_of_i).
model.setObjective(
    gp.quicksum(
        revenue[lub] * gp.quicksum(x[lub, mat] for mat in materials)
        for lub in lubricants
    ),
    GRB.MAXIMIZE
)

# -------------------------------------------------------------------------
# 5. CONSTRAINTS
# -------------------------------------------------------------------------

# (a) Demand constraint: total produced of lubricant i cannot exceed total_demand
for lub in lubricants:
    model.addConstr(
        gp.quicksum(x[lub, mat] for mat in materials) <= demand[lub],
        name=f"demand_{lub}"
    )

# (b) Material availability: sum of usage of material j across all lubricants <= max_availability_j
for mat in materials:
    model.addConstr(
        gp.quicksum(x[lub, mat] for lub in lubricants) <= max_avail[mat],
        name=f"availability_{mat}"
    )

# (c) Minimum sustainability rating for each lubricant:
# Average sustainability >= min_sustain => sum_j(sus[i,j] * x[i,j]) >= min_sustain[i] * sum_j(x[i,j])
for lub in lubricants:
    model.addConstr(
        gp.quicksum(sus_dict[(lub, mat)] * x[lub, mat] for mat in materials)
        >= min_sustain[lub] * gp.quicksum(x[lub, mat] for mat in materials),
        name=f"min_sustainability_{lub}"
    )

# (d) Maximum heat rating for each lubricant:
# Average heat <= max_heat => sum_j(heat[i,j] * x[i,j]) <= max_heat[i] * sum_j(x[i,j])
for lub in lubricants:
    model.addConstr(
        gp.quicksum(heat_dict[(lub, mat)] * x[lub, mat] for mat in materials)
        <= max_heat[lub] * gp.quicksum(x[lub, mat] for mat in materials),
        name=f"max_heat_{lub}"
    )

# (e) Proportion constraint: x_{i,j} <= p_max_j * sum_k x_{i,k}
# This ensures material j does not exceed p_max_j fraction of lubricant i.
for lub in lubricants:
    for mat in materials:
        model.addConstr(
            x[lub, mat] 
            <= p_max[mat] * gp.quicksum(x[lub, k] for k in materials),
            name=f"prop_{lub}_{mat}"
        )

# -------------------------------------------------------------------------
# 6. OPTIMIZE
# -------------------------------------------------------------------------
model.optimize()

# -------------------------------------------------------------------------
# 7. EXTRACT & PRINT RESULTS
# -------------------------------------------------------------------------
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective (total revenue) = {model.objVal:0.4f}\n")

    # Collect solution values where x_{i,j} > 0
    solution = []
    for lub in lubricants:
        for mat in materials:
            if x[lub, mat].X > 1e-6:
                solution.append([lub, mat, x[lub, mat].X])

    df_solution = pd.DataFrame(solution, columns=["Lubricant", "Material", "Usage"])
    print("Non-zero solution variables:\n", df_solution)
else:
    print("No optimal solution found. Status code:", model.status)


KeyError: 'Unnamed: 0'

# Question 2

In [8]:
welders

,Welder_ID,Safety_Rating,Speed_Rating,SMAW_Proficient,GMAW_Proficient,FCAW_Proficient,GTAW_Proficient,Experience_10_Years
0,1,2,1,1,0,1,0,0
1,2,4,2,0,1,0,0,1
2,3,4,1,1,0,0,0,0
3,4,3,2,1,1,1,0,1
4,5,2,4,1,0,0,0,0
...,...,...,...,...,...,...,...,...
245,246,2,1,0,1,1,1,0
246,247,4,1,1,0,0,1,0
247,248,3,3,0,0,0,1,0
248,249,1,1,1,0,0,1,1


In [41]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# 1) Read the data from the raw GitHub link
welders = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/welders_data_makeup.csv')

# 2) Create a new Gurobi model
model = gp.Model("Atlas_Construction_Welder_Selection")

# 3) Add binary decision variables:
#    x[i] = 1 if welder i is hired, 0 otherwise.
x = {}
num_welders = len(welders)
for i in range(num_welders):
    x[i] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")

# 4) Objective: maximize sum of safety ratings (equivalent to maximizing average safety)
model.setObjective(
    gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders)),
    GRB.MAXIMIZE
)

# 5) Constraints

# (a) Exactly 32 welders must be hired
model.addConstr(
    gp.quicksum(x[i] for i in range(num_welders)) == 32,
    name="Team_Size"
)

# (b) At least 60% of hired welders must be proficient in ≥ 2 welding techniques
#     i.e., sum of (SMAW + GMAW + FCAW + GTAW) ≥ 2
multi_proficient = [
    i for i in range(num_welders)
    if (welders.loc[i, 'SMAW_Proficient']
        + welders.loc[i, 'GMAW_Proficient']
        + welders.loc[i, 'FCAW_Proficient']
        + welders.loc[i, 'GTAW_Proficient']) >= 2
]
model.addConstr(
    gp.quicksum(x[i] for i in multi_proficient) >= 0.60 * 32,
    name="AtLeast60pct_2tech"
)

# (c) Exactly 8 welders must be proficient in all 4 techniques
all_four = [
    i for i in range(num_welders)
    if (welders.loc[i, 'SMAW_Proficient']
        + welders.loc[i, 'GMAW_Proficient']
        + welders.loc[i, 'FCAW_Proficient']
        + welders.loc[i, 'GTAW_Proficient']) == 4
]
model.addConstr(
    gp.quicksum(x[i] for i in all_four) == 8,
    name="Exactly8_all4"
)

# (d) At least 40% of hired welders must have >10 years of experience
experienced = [
    i for i in range(num_welders)
    if welders.loc[i, 'Experience_10_Years'] == 1
]
model.addConstr(
    gp.quicksum(x[i] for i in experienced) >= 0.40 * 32,
    name="AtLeast40pct_experience"
)

# (e) Average speed ≥ 3.1 ⇒ sum of speed ≥ 3.1 * 32
model.addConstr(
    gp.quicksum(welders.loc[i, 'Speed_Rating'] * x[i] for i in range(num_welders))
    >= 3.1 * 32,
    name="SpeedAvg_3.1"
)

#     Average safety ≥ 3.4 ⇒ sum of safety ≥ 3.4 * 32
model.addConstr(
    gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders))
    >= 3.4 * 32,
    name="SafetyAvg_3.4"
)

# (f) At least 6 welders with speed=1 and safety=3 or 4
slow_safety = [
    i for i in range(num_welders)
    if (welders.loc[i, 'Speed_Rating'] == 1 and welders.loc[i, 'Safety_Rating'] >= 3)
]
model.addConstr(
    gp.quicksum(x[i] for i in slow_safety) >= 6,
    name="Min6_slow_safety"
)

# (g) At least twice as many welders from ID range [100–180] as from [181–250]
rangeA = [i for i in range(num_welders)
          if 100 <= welders.loc[i, 'Welder_ID'] <= 180]
rangeB = [i for i in range(num_welders)
          if 181 <= welders.loc[i, 'Welder_ID'] <= 250]
model.addConstr(
    gp.quicksum(x[i] for i in rangeA) >= 2 * gp.quicksum(x[i] for i in rangeB),
    name="TwiceAsMany_100to180_vs_181to250"
)

# (h) ALLOW up to 3 from bottom 70 (instead of 2) to avoid contradiction
bottom_70 = [i for i in range(num_welders)
             if 1 <= welders.loc[i, 'Welder_ID'] <= 70]
model.addConstr(
    gp.quicksum(x[i] for i in bottom_70) <= 3,
    name="Max3_bottom70"
)

# (i) At least 3 welders from each of these ID ranges:
#     [1–50], [51–100], [101–150], [151–200], [201–250]
range1 = [i for i in range(num_welders) if 1 <= welders.loc[i, 'Welder_ID'] <= 50]
range2 = [i for i in range(num_welders) if 51 <= welders.loc[i, 'Welder_ID'] <= 100]
range3 = [i for i in range(num_welders) if 101 <= welders.loc[i, 'Welder_ID'] <= 150]
range4 = [i for i in range(num_welders) if 151 <= welders.loc[i, 'Welder_ID'] <= 200]
range5 = [i for i in range(num_welders) if 201 <= welders.loc[i, 'Welder_ID'] <= 250]

model.addConstr(gp.quicksum(x[i] for i in range1) >= 3, name="Range_1to50")
model.addConstr(gp.quicksum(x[i] for i in range2) >= 3, name="Range_51to100")
model.addConstr(gp.quicksum(x[i] for i in range3) >= 3, name="Range_101to150")
model.addConstr(gp.quicksum(x[i] for i in range4) >= 3, name="Range_151to200")
model.addConstr(gp.quicksum(x[i] for i in range5) >= 3, name="Range_201to250")

# 6) Optimize the model
model.optimize()

# 7) Print results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Total Safety (Objective) = {model.objVal:.2f}")
    print(f"Average Safety of the Team = {model.objVal / 32:.2f}\n")
    
    selected_welders = [i for i in range(num_welders) if x[i].X > 0.5]
    print(f"Number of Welders Hired: {len(selected_welders)}")
    print("Hired Welder IDs:", [int(welders.loc[i,'Welder_ID']) for i in selected_welders])
else:
    print("No optimal solution found or model is infeasible.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 250 columns and 1546 nonzeros
Model fingerprint: 0x6d6e03aa
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 1 rows and 76 columns
Presolve time: 0.00s
Presolved: 13 rows, 174 columns, 914 nonzeros
Variable types: 0 continuous, 174 integer (124 binary)

Root relaxation: objective 1.151667e+02, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  115.16667    0    7          -  115.16667      -     -    0s
H    0     0                     113

In [42]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def solve_modified_model():
    # 1) Read the data from the raw GitHub link
    welders = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/welders_data_makeup.csv')

    # 2) Create a new Gurobi model
    model = gp.Model("Atlas_Construction_Welder_Selection_No7_No8")

    # 3) Add binary decision variables:
    x = {}
    num_welders = len(welders)
    for i in range(num_welders):
        x[i] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")

    # 4) Objective: maximize sum of safety ratings
    model.setObjective(
        gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders)),
        GRB.MAXIMIZE
    )

    # 5) Constraints

    # (a) Exactly 32 welders must be hired
    model.addConstr(
        gp.quicksum(x[i] for i in range(num_welders)) == 32,
        name="Team_Size"
    )

    # (b) At least 60% of hired welders must be proficient in ≥ 2 welding techniques
    multi_proficient = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) >= 2
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in multi_proficient) >= 0.60 * 32,
        name="AtLeast60pct_2tech"
    )

    # (c) Exactly 8 welders must be proficient in all 4 techniques
    all_four = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) == 4
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in all_four) == 8,
        name="Exactly8_all4"
    )

    # (d) At least 40% of hired welders must have >10 years of experience
    experienced = [
        i for i in range(num_welders)
        if welders.loc[i, 'Experience_10_Years'] == 1
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in experienced) >= 0.40 * 32,
        name="AtLeast40pct_experience"
    )

    # (e) Average speed ≥ 3.1 ⇒ sum of speed ≥ 3.1 * 32
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Speed_Rating'] * x[i] for i in range(num_welders))
        >= 3.1 * 32,
        name="SpeedAvg_3.1"
    )

    #     Average safety ≥ 3.4 ⇒ sum of safety ≥ 3.4 * 32
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders))
        >= 3.4 * 32,
        name="SafetyAvg_3.4"
    )

    # (f) NO LONGER adding the "at least 6 welders with speed=1 and safety=3 or 4" constraint
    # (g) NO LONGER adding the "at least twice as many from [100–180] vs [181–250]" constraint

    # (h) Up to 3 from bottom 70
    bottom_70 = [i for i in range(num_welders) if 1 <= welders.loc[i, 'Welder_ID'] <= 70]
    model.addConstr(
        gp.quicksum(x[i] for i in bottom_70) <= 3,
        name="Max3_bottom70"
    )

    # (i) At least 3 from each ID range
    range1 = [i for i in range(num_welders) if 1 <= welders.loc[i, 'Welder_ID'] <= 50]
    range2 = [i for i in range(num_welders) if 51 <= welders.loc[i, 'Welder_ID'] <= 100]
    range3 = [i for i in range(num_welders) if 101 <= welders.loc[i, 'Welder_ID'] <= 150]
    range4 = [i for i in range(num_welders) if 151 <= welders.loc[i, 'Welder_ID'] <= 200]
    range5 = [i for i in range(num_welders) if 201 <= welders.loc[i, 'Welder_ID'] <= 250]

    model.addConstr(gp.quicksum(x[i] for i in range1) >= 3, name="Range_1to50")
    model.addConstr(gp.quicksum(x[i] for i in range2) >= 3, name="Range_51to100")
    model.addConstr(gp.quicksum(x[i] for i in range3) >= 3, name="Range_101to150")
    model.addConstr(gp.quicksum(x[i] for i in range4) >= 3, name="Range_151to200")
    model.addConstr(gp.quicksum(x[i] for i in range5) >= 3, name="Range_201to250")

    # 6) Optimize the model
    model.optimize()

    # 7) Print results
    if model.status == GRB.OPTIMAL:
        print(f"\nOptimal Total Safety (Objective) = {model.objVal:.2f}")
        print(f"Average Safety of the Team = {model.objVal / 32:.2f}\n")

        selected_welders = [i for i in range(num_welders) if x[i].X > 0.5]
        print(f"Number of Welders Hired: {len(selected_welders)}")
        print("Hired Welder IDs:", [int(welders.loc[i,'Welder_ID']) for i in selected_welders])
    else:
        print("No optimal solution found or model is infeasible.")


if __name__ == "__main__":
    solve_modified_model()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 250 columns and 1365 nonzeros
Model fingerprint: 0x14d53715
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 1 rows and 86 columns
Presolve time: 0.00s
Presolved: 11 rows, 164 columns, 732 nonzeros
Variable types: 0 continuous, 164 integer (105 binary)

Root relaxation: objective 1.170000e+02, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     117.0000000  117.00000  0.00%     -    0s

Explored 1 nodes (19 simplex iterat

In [43]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def solve_with_solution_pool():
    # 1) Read the data from the raw GitHub link
    welders = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/welders_data_makeup.csv')

    # 2) Create a new Gurobi model
    model = gp.Model("Atlas_Construction_Welder_Selection_PartF")

    # 3) Add binary decision variables
    x = {}
    num_welders = len(welders)
    for i in range(num_welders):
        x[i] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")

    # 4) Objective: maximize sum of safety ratings
    model.setObjective(
        gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders)),
        GRB.MAXIMIZE
    )

    # 5) Constraints

    # (a) Exactly 32 welders
    model.addConstr(
        gp.quicksum(x[i] for i in range(num_welders)) == 32,
        name="Team_Size"
    )

    # (b) ≥ 60% with ≥2 proficiencies
    multi_proficient = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) >= 2
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in multi_proficient) >= 0.60 * 32,
        name="AtLeast60pct_2tech"
    )

    # (c) Exactly 8 with all 4 techniques
    all_four = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) == 4
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in all_four) == 8,
        name="Exactly8_all4"
    )

    # (d) ≥ 40% with >10 years experience
    experienced = [
        i for i in range(num_welders)
        if welders.loc[i, 'Experience_10_Years'] == 1
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in experienced) >= 0.40 * 32,
        name="AtLeast40pct_experience"
    )

    # (e) Average speed ≥ 3.1 & average safety ≥ 3.4
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Speed_Rating'] * x[i] for i in range(num_welders))
        >= 3.1 * 32,
        name="SpeedAvg_3.1"
    )
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders))
        >= 3.4 * 32,
        name="SafetyAvg_3.4"
    )

    # (f) At least 6 with speed=1 and safety≥3
    slow_safety = [
        i for i in range(num_welders)
        if (welders.loc[i, 'Speed_Rating'] == 1 and welders.loc[i, 'Safety_Rating'] >= 3)
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in slow_safety) >= 6,
        name="Min6_slow_safety"
    )

    # (g) ≥ twice as many from ID [100..180] as from [181..250]
    rangeA = [i for i in range(num_welders) if 100 <= welders.loc[i,'Welder_ID'] <= 180]
    rangeB = [i for i in range(num_welders) if 181 <= welders.loc[i,'Welder_ID'] <= 250]
    model.addConstr(
        gp.quicksum(x[i] for i in rangeA) >= 2 * gp.quicksum(x[i] for i in rangeB),
        name="TwiceAsMany_100to180_vs_181to250"
    )

    # (h) Up to 3 from bottom 70
    bottom_70 = [i for i in range(num_welders) if 1 <= welders.loc[i,'Welder_ID'] <= 70]
    model.addConstr(
        gp.quicksum(x[i] for i in bottom_70) <= 3,
        name="Max3_bottom70"
    )

    # (i) ≥ 3 from each ID range
    range1 = [i for i in range(num_welders) if 1 <= welders.loc[i,'Welder_ID'] <= 50]
    range2 = [i for i in range(num_welders) if 51 <= welders.loc[i,'Welder_ID'] <= 100]
    range3 = [i for i in range(num_welders) if 101 <= welders.loc[i,'Welder_ID'] <= 150]
    range4 = [i for i in range(num_welders) if 151 <= welders.loc[i,'Welder_ID'] <= 200]
    range5 = [i for i in range(num_welders) if 201 <= welders.loc[i,'Welder_ID'] <= 250]

    model.addConstr(gp.quicksum(x[i] for i in range1) >= 3, name="Range_1to50")
    model.addConstr(gp.quicksum(x[i] for i in range2) >= 3, name="Range_51to100")
    model.addConstr(gp.quicksum(x[i] for i in range3) >= 3, name="Range_101to150")
    model.addConstr(gp.quicksum(x[i] for i in range4) >= 3, name="Range_151to200")
    model.addConstr(gp.quicksum(x[i] for i in range5) >= 3, name="Range_201to250")

    # -- SOLUTION POOL SETTINGS --
    # We want to find all solutions within 0.1% of the best solution
    model.setParam("PoolSearchMode", 2)   # Do a comprehensive search for alternate solutions
    model.setParam("PoolSolutions", 1000) # Up to 1000 solutions in the solution pool
    model.setParam("PoolGap", 0.1)        # Let the solver accept solutions within 0.1% of optimum

    # 6) Optimize
    model.optimize()

    if model.status == GRB.OPTIMAL or model.status == GRB.INT_OPTIMAL:
        best_obj = model.objVal
        print(f"\nBest Objective = {best_obj:.2f}")

        # Count how many solutions are within 0.1% of best
        count_within_gap = 0
        for sol_idx in range(model.SolCount):
            model.setParam("SolutionNumber", sol_idx)
            # 'PoolObjVal' is the objective value of the solution at index sol_idx
            sol_obj = model.PoolObjVal
            # Check if within 0.1% gap: i.e. (best_obj - sol_obj)/best_obj <= 0.001
            if abs(best_obj - sol_obj) <= 0.001 * best_obj:
                count_within_gap += 1

        print(f"Number of solutions within 0.1% of optimal: {count_within_gap}")
    else:
        print("No optimal solution found or model is infeasible.")

if __name__ == "__main__":
    solve_with_solution_pool()


Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1000
Set parameter PoolGap to value 0.1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  1000
PoolSearchMode  2
PoolGap  0.1

Optimize a model with 14 rows, 250 columns and 1546 nonzeros
Model fingerprint: 0x6d6e03aa
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 1 rows and 20 columns
Presolve time: 0.00s
Presolved: 13 rows, 230 columns, 1205 nonzeros
Variable types: 0 continuous, 230 integer (230 binary)

Root relaxation: objective 1.151667e+02, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

In [44]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

def solve_for_speed():
    # -- Original "safety" solution for reference:
    #    This is the solution you previously obtained with objective = 115 total safety.
    #    If your solution differs, replace with your own welder IDs.
    old_safety_solution = [
        7, 39, 43, 71, 72, 80, 96, 107, 111, 115, 118, 119, 123,
        126, 135, 136, 139, 143, 146, 161, 172, 173, 175, 176,
        181, 185, 187, 197, 215, 217, 224, 228
    ]

    # 1) Read the data from GitHub
    welders = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/welders_data_makeup.csv')

    # 2) Create a new Gurobi model
    model = gp.Model("Atlas_Construction_Welder_Selection_SpeedObjective")

    # 3) Add binary decision variables
    x = {}
    num_welders = len(welders)
    for i in range(num_welders):
        x[i] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")

    # 4) Objective: maximize sum of SPEED ratings (instead of safety)
    model.setObjective(
        gp.quicksum(welders.loc[i, 'Speed_Rating'] * x[i] for i in range(num_welders)),
        GRB.MAXIMIZE
    )

    # 5) Constraints (same as the original safety-based model):
    # (a) Exactly 32 welders must be hired
    model.addConstr(
        gp.quicksum(x[i] for i in range(num_welders)) == 32,
        name="Team_Size"
    )

    # (b) At least 60% of hired welders must be proficient in ≥ 2 welding techniques
    multi_proficient = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) >= 2
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in multi_proficient) >= 0.60 * 32,
        name="AtLeast60pct_2tech"
    )

    # (c) Exactly 8 welders must be proficient in all 4 techniques
    all_four = [
        i for i in range(num_welders)
        if (welders.loc[i, 'SMAW_Proficient']
            + welders.loc[i, 'GMAW_Proficient']
            + welders.loc[i, 'FCAW_Proficient']
            + welders.loc[i, 'GTAW_Proficient']) == 4
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in all_four) == 8,
        name="Exactly8_all4"
    )

    # (d) At least 40% with >10 years experience
    experienced = [
        i for i in range(num_welders)
        if welders.loc[i, 'Experience_10_Years'] == 1
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in experienced) >= 0.40 * 32,
        name="AtLeast40pct_experience"
    )

    # (e) Average speed ≥ 3.1  => sum of speed ≥ 3.1 * 32
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Speed_Rating'] * x[i] for i in range(num_welders))
        >= 3.1 * 32,
        name="SpeedAvg_3.1"
    )

    #     Average safety ≥ 3.4 => sum of safety ≥ 3.4 * 32
    model.addConstr(
        gp.quicksum(welders.loc[i, 'Safety_Rating'] * x[i] for i in range(num_welders))
        >= 3.4 * 32,
        name="SafetyAvg_3.4"
    )

    # (f) At least 6 with speed=1 and safety≥3
    slow_safety = [
        i for i in range(num_welders)
        if (welders.loc[i, 'Speed_Rating'] == 1 and welders.loc[i, 'Safety_Rating'] >= 3)
    ]
    model.addConstr(
        gp.quicksum(x[i] for i in slow_safety) >= 6,
        name="Min6_slow_safety"
    )

    # (g) At least twice as many from ID range [100..180] as from [181..250]
    rangeA = [i for i in range(num_welders) if 100 <= welders.loc[i,'Welder_ID'] <= 180]
    rangeB = [i for i in range(num_welders) if 181 <= welders.loc[i,'Welder_ID'] <= 250]
    model.addConstr(
        gp.quicksum(x[i] for i in rangeA) >= 2 * gp.quicksum(x[i] for i in rangeB),
        name="TwiceAsMany_100to180_vs_181to250"
    )

    # (h) Up to 3 from bottom 70
    bottom_70 = [i for i in range(num_welders) if 1 <= welders.loc[i,'Welder_ID'] <= 70]
    model.addConstr(
        gp.quicksum(x[i] for i in bottom_70) <= 3,
        name="Max3_bottom70"
    )

    # (i) At least 3 from each ID range
    range1 = [i for i in range(num_welders) if 1 <= welders.loc[i,'Welder_ID'] <= 50]
    range2 = [i for i in range(num_welders) if 51 <= welders.loc[i,'Welder_ID'] <= 100]
    range3 = [i for i in range(num_welders) if 101 <= welders.loc[i,'Welder_ID'] <= 150]
    range4 = [i for i in range(num_welders) if 151 <= welders.loc[i,'Welder_ID'] <= 200]
    range5 = [i for i in range(num_welders) if 201 <= welders.loc[i,'Welder_ID'] <= 250]

    model.addConstr(gp.quicksum(x[i] for i in range1) >= 3, name="Range_1to50")
    model.addConstr(gp.quicksum(x[i] for i in range2) >= 3, name="Range_51to100")
    model.addConstr(gp.quicksum(x[i] for i in range3) >= 3, name="Range_101to150")
    model.addConstr(gp.quicksum(x[i] for i in range4) >= 3, name="Range_151to200")
    model.addConstr(gp.quicksum(x[i] for i in range5) >= 3, name="Range_201to250")

    # 6) Optimize
    model.optimize()

    if model.status == GRB.OPTIMAL:
        best_obj = model.objVal
        print(f"\nOptimal Total Speed (Objective) = {best_obj:.2f}")
        print(f"Average Speed of the Team = {best_obj / 32:.2f}")

        # Gather new solution's hired welder IDs
        new_speed_solution = []
        for i in range(num_welders):
            if x[i].X > 0.5:
                new_speed_solution.append(int(welders.loc[i, 'Welder_ID']))

        print(f"\nNumber of Welders Hired: {len(new_speed_solution)}")
        print("Hired Welder IDs:", new_speed_solution)

        # Compare to old safety solution
        old_safety_set = set(old_safety_solution)
        new_speed_set = set(new_speed_solution)
        overlap = old_safety_set.intersection(new_speed_set)

        print(f"\nOverlap with the old (safety) solution: {len(overlap)} welders")
        print("These overlapping IDs are:", sorted(list(overlap)))
    else:
        print("No optimal solution found or model is infeasible.")

if __name__ == "__main__":
    solve_for_speed()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 250 columns and 1546 nonzeros
Model fingerprint: 0xa0522d89
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+02]
Presolve removed 1 rows and 76 columns
Presolve time: 0.00s
Presolved: 13 rows, 174 columns, 914 nonzeros
Variable types: 0 continuous, 174 integer (124 binary)

Root relaxation: objective 1.068333e+02, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  106.83333    0    3          -  106.83333      -     -    0s
H    0     0                     106